In [1]:
import tensorflow as tf
import pandas as pd
from config import *
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.preprocessing import sequence
import numpy as np
from tqdm import tqdm
from IPython.display import clear_output

tf.enable_eager_execution()
tf.test.is_gpu_available()

True

In [ ]:
### Hyperparameter
###################
SAVE_PATH = '/home/huangzc/competition/tencent/model_ckpt/simple_nn/model.ckpt'
EMB_SIZE = 50
BATCH_SIZE = 10240
EPOCHS = 100

In [ ]:
### Read data
tr_ad_df = pd.read_pickle(TRAIN_DIR+AD_PATH)
ts_ad_df = pd.read_pickle(TEST_DIR+AD_PATH)

tr_clk_list_df = pd.read_pickle(TRAIN_DIR+CLK_LIST_PATH)
ts_clk_list_df = pd.read_pickle(TEST_DIR+CLK_LIST_PATH)

tr_user_df = pd.read_pickle(TRAIN_DIR+USER_PATH)
tr_user_df = tr_user_df.groupby(['user_id']).agg({'age': 'first', 'gender': 'first'}).reset_index()

assert tr_user_df['user_id'].values.tolist() == tr_clk_list_df['user_id'].values.tolist()
grid_df = pd.concat([tr_clk_list_df, tr_user_df[['age']]], axis=1)

In [4]:
feature_columns = ['clk_list']
label_column = 'age'

grid_df[label_column] = grid_df[label_column] - 1

In [5]:
### Split into train and valid dataset
train = grid_df[feature_columns+[label_column]]
test = ts_clk_list_df[feature_columns]

train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

### x, y dataframe
x_train, y_train = train[feature_columns], train[label_column]
x_val, y_val = val[feature_columns], val[label_column]
x_test = test

720000 train examples
180000 validation examples
1000000 test examples


In [6]:
sentence_size = int(grid_df['clk_list'].map(lambda x: len(x)).quantile(0.9))
print('choose sentences max len: %d' % sentence_size)

choose sentences max len: 66


In [7]:
print("Pad sequences (samples x time)")

### feature_columns[0] means click list
x_train = sequence.pad_sequences(x_train[feature_columns[0]],
                                 maxlen=sentence_size, 
                                 padding='post', 
                                 truncating='post',
                                 dtype='int64',
                                 value=0
                                 )
x_val = sequence.pad_sequences(x_val[feature_columns[0]],
                                 maxlen=sentence_size, 
                                 padding='post', 
                                 truncating='post',
                                 dtype='int64',
                                 value=0
                                 )
x_test = sequence.pad_sequences(x_test[feature_columns[0]], 
                                maxlen=sentence_size, 
                                padding='post',
                                truncating='post',
                                dtype='int64',
                                value=0
                               )
print("x_train shape:", x_train.shape)
print("x_val shape:", x_val.shape)
print("x_test shape:", x_test.shape)

Pad sequences (samples x time)
x_train shape: (720000, 66)
x_val shape: (180000, 66)
x_test shape: (1000000, 66)


In [8]:
### Get tf dataset
def get_train_ds(x, y): 
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=len(x))
    dataset = dataset.batch(BATCH_SIZE)
#     dataset = dataset.repeat(EPOCHS)
    return dataset

def get_test_ds(x, ): 
    dataset = tf.data.Dataset.from_tensor_slices((x, )).batch(BATCH_SIZE)
    return dataset

train_ds = get_train_ds(x_train, y_train.values)
valid_ds = get_train_ds(x_val, y_val.values)
test_ds = get_test_ds(x_test)

In [9]:
vocab_size = max(pd.concat([tr_ad_df, ts_ad_df], axis=0)['creative_id'].unique().tolist()) + 1 ### padding 0 need add 1

In [ ]:
# weights = np.load('/home/huangzc/competition/tencent/data/train_preliminary/gensim_dict.npy')

In [ ]:
### Construct Model
#################################
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
#         self.embedding = tf.keras.layers.Embedding(vocab_size, EMB_SIZE, weights=[weights])
        self.embedding = tf.keras.layers.Embedding(vocab_size, EMB_SIZE)
        self.pool1D = tf.keras.layers.GlobalAveragePooling1D()
        self.dense1 = tf.keras.layers.Dense(16, activation=tf.nn.leaky_relu)
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.pool1D(x)
        x = self.dense1(x)
        return self.dense2(x)

model = MyModel()

In [ ]:
# Restore the weights
model.load_weights(SAVE_PATH)

In [ ]:
### Loss & Metric
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

valid_loss = tf.keras.metrics.Mean(name='valid_loss')
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')

In [ ]:
@tf.function
def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [ ]:
@tf.function
def valid_step(features, labels):
    predictions = model(features)
    v_loss = loss_object(labels, predictions)

    valid_loss(v_loss)
    valid_accuracy(labels, predictions)

In [ ]:
for epoch in range(EPOCHS):
  # 在下一个epoch开始时，重置评估指标
    train_loss.reset_states()
    train_accuracy.reset_states()
    valid_loss.reset_states()
    valid_accuracy.reset_states()

    CNT = 0
    for features, labels in train_ds:
#         clear_output(wait=True)
        train_step(features, labels)
        CNT += 1
        print('batch: %d, batch loss: %.4f' % (CNT, train_loss.result()))    
        
    for val_features, val_labels in valid_ds:
        valid_step(val_features, val_labels)

    template = 'Epoch {}, Loss: {:.2f}, Accuracy: {:.2f}%, Valid Loss: {:.2f}, Valid Accuracy: {:.2f}%'
    print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         valid_loss.result(),
                         valid_accuracy.result()*100))

In [ ]:
# 保存权重
model.save_weights(SAVE_PATH)